# Uso avanzado de bases de datos NoSQL


In [49]:
from pymongo import MongoClient

# Conexión al servidor local
client = MongoClient('localhost', 27017)

# Selección de la base de datos
db = client['localesMadrid']
# Selección de la colección
collection = db['madridDbLocalesB']

cursor = collection.find({},{'inspeccionar':1}).limit(5)

for doc in cursor:
    print(doc)


{'_id': 7, 'inspeccionar': False}
{'_id': 32, 'inspeccionar': False}
{'_id': 33, 'inspeccionar': False}
{'_id': 34}
{'_id': 36}


#### 1.	Los locales del barrio Guindalera de Salamanca, por motivos de la desescalada, no podrán abrir y tendrán que permanecer cerrados. Cambiad la situación del local a Cerrado utilizando el id correspondiente. Recordad que debéis constatar los tipos de situaciones que existan y seguir con la misma codificación. A estos mismos locales, cambiad la situación de la terraza a cerrada siguiendo las mismas premisas anteriores.

In [50]:
filtro = {"desc_distrito_local": "SALAMANCA", 'desc_barrio_local':'GUINDALERA'}
# Actualización: cambiar estado del local (solo descripción) y de la terraza (id = 8)
nuevos_valores = {
    "$set": {
        "desc_situacion_local": "Cerrado",
        "id_situacion_terraza": 8,        
        "desc_situacion_terraza": "Cerrada"
    }
}

resultado = collection.update_many(filtro, nuevos_valores)

cursor = collection.find(filtro).limit(5)

for doc in cursor:
    print(doc)

{'_id': 33, 'id_local': 270403150, 'id_distrito_local': 4, 'desc_distrito_local': 'SALAMANCA', 'desc_barrio_local': 'GUINDALERA', 'clase_vial_edificio': 'AVENIDA', 'num_edificio': 2, 'Cod_Postal': 28028, 'coordenada_x_local': '442586,58', 'coordenada_y_local': '4476524,51', 'desc_situacion_local': 'Cerrado', 'Nombre': 'HONTANARES', 'id_periodo_terraza': 1, 'desc_periodo_terraza': 'Anual', 'id_situacion_terraza': 8, 'desc_situacion_terraza': 'Cerrada', 'Superficie_ES': '21,06', 'DESC_CLASE': 'AVENIDA', 'DESC_NOMBRE': 'AMERICA', 'num_terraza': 2, 'cal_terraza': '', 'desc_ubicacion_terraza': 'Acera', 'hora_ini_LJ_es': '10:00:00', 'hora_fin_LJ_es': '1:00:00', 'hora_ini_LJ_ra': '10:00:00', 'hora_fin_LJ_ra': '0:00:00', 'hora_ini_VS_es': '10:00:00', 'hora_fin_VS_es': '2:30:00', 'hora_ini_VS_ra': '10:00:00', 'hora_fin_VS_ra': '0:00:00', 'mesas_aux_es': 0, 'mesas_aux_ra': 0, 'mesas_es': 9, 'mesas_ra': 9, 'sillas_es': 27, 'sillas_ra': 0, 'inspeccionar': False}
{'_id': 34, 'id_local': 270078567, 


2.	A todas las terrazas que se ubiquen en la acera, añadid un campo llamado inspeccionar y estableced el siguiente valor:
Si dispone de más de 10 mesas, true.
Si dispone de menos de 10 mesas, false.


In [51]:
filtro_2 = {'desc_ubicacion_terraza': 'Acera'}

update_2 = [
    {
        "$set": {
            "inspeccionar": {
                "$cond": {
                    "if": {"$gt": ["$mesas_es", 10]},  # si mesas_es > 10
                    "then": True,                # entonces
                    "else": False             # sino
                }
            }
        }
    }
]

# Aplicar update_many
resultado = collection.update_many(filtro_2, update_2)
print("Documentos modificados:", resultado.modified_count)




Documentos modificados: 0


In [58]:
cursor = collection.find(filtro_2, {"_id": 1, "mesas_es": 1, "inspeccionar": 1}).limit(5)

for doc in cursor:
    print(doc)

{'_id': 7, 'mesas_es': 5, 'inspeccionar': False}
{'_id': 32, 'mesas_es': 10, 'inspeccionar': False}
{'_id': 33, 'mesas_es': 9, 'inspeccionar': False}
{'_id': 40, 'mesas_es': 8, 'inspeccionar': False}
{'_id': 41, 'mesas_es': 6, 'inspeccionar': False}


3.	A las terrazas que se deban inspeccionar, añadid 2 mesas más auxiliares y 8 sillas más disponibles (tanto en período estacional como el resto del año).

In [56]:


# Contador de documentos modificados
modificados = 0

# Iterar sobre todas las terrazas que se deben inspeccionar
for doc in collection.find({"inspeccionar": True}):
    # Asegurarse de que los campos sean números; si no existen o son None, poner 0
    mesas_es = int(doc.get("mesas_es") or 0)
    mesas_ra = int(doc.get("mesas_ra") or 0)
    sillas_es = int(doc.get("sillas_es") or 0)
    
    # Actualizar sumando mesas y sillas
    resultado = collection.update_one(
        {"_id": doc["_id"]},
        {"$set": {
            "mesas_es": mesas_es +2 ,
            "mesas_ra": mesas_ra +2,
            "sillas_es": sillas_es +8
        }}
    )
    
    if resultado.modified_count > 0:
        modificados += 1

print("Documentos modificados:", modificados)

# Mostrar 5 documentos actualizados como verificación
cursor = collection.find(
    {"inspeccionar": True},
    {"_id": 1, "mesas_es": 1, "mesas_ra":1, "sillas_es":1}
).limit(5)

for doc in cursor:
    print(doc)



Documentos modificados: 750
{'_id': 51, 'mesas_es': 18, 'mesas_ra': 4, 'sillas_es': 51}
{'_id': 67, 'mesas_es': 14, 'mesas_ra': 14, 'sillas_es': 55}
{'_id': 73, 'mesas_es': 32, 'mesas_ra': 32, 'sillas_es': 128}
{'_id': 78, 'mesas_es': 13, 'mesas_ra': 2, 'sillas_es': 52}
{'_id': 80, 'mesas_es': 18, 'mesas_ra': 12, 'sillas_es': 64}


#### 4.	A las terrazas que no deban ser inspeccionadas, añadid el campo estado con el valor:
#### 1 si el número de sillas es menor que 10.
#### 2 si el número de sillas está entre 10 y 20.
#### 3 si cuenta con más de 20 sillas.